<a href="https://colab.research.google.com/github/AluminumShark/2024_Deep_Learning_with_Pytorch/blob/main/01_MINIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Import Modules

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
import numpy as np
torch.manual_seed(681520)

In [24]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 20
BATCH_SIZE = 128
lr = 0.001

In [8]:
# Load and Normalize Datasets

trainset = MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=trainset, batch_size=len(trainset), shuffle=True)

data = next(iter(train_loader))

mean = data[0].mean()
std = data[0].std()

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean, std)])

trainset = MNIST(root='./data', train=True, transform=transform, download=True)
testset = MNIST(root='./data', train=True, transform=transform, download=True)

In [11]:
# Creat a Sequential Model

model = nn.Sequential(
    nn.Flatten(),

    nn.Linear(28*28, 25),
    nn.Dropout(0.2),
    nn.ReLU(),

    nn.Linear(25, 10),
    nn.Dropout(0.2),
    nn.ReLU()

)

for module in model.modules():
    if isinstance(module, nn.Linear):
        nn.init.kaiming_uniform_(module.weight, nonlinearity='relu')
        nn.init.constant_(module.bias, 0)

In [23]:
optimizer = torch.optim.Adam(model.parameters(), lr)
loss_fn = nn.CrossEntropyLoss()

model.to(device)

train_loader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(testset, batch_size=BATCH_SIZE, shuffle=True)


for i in range(EPOCHS):
  model.train()
  train_loss = 0.0
  train_acc = 0
  train_batches = 0

  # Start Training

  for inputs, targets in train_loader:
    one_hot_targets = nn.functional.one_hot(targets, num_classes=10).float()
    inputs, targets, one_hot_targets = inputs.to(device), targets.to(device), one_hot_targets.to(device)

    optimizer.zero_grad()

    # Forward pass
    outputs = model(inputs)
    loss = loss_fn(outputs, one_hot_targets)

    # Caculate Accuracy
    _, predicted = torch.max(outputs.data, 1)
    train_acc += (predicted == targets).sum().item()
    train_batches += 1
    train_loss += loss.item()

    # Backward pass and Update
    loss.backward()
    optimizer.step()

  train_loss /= train_batches
  train_acc /= train_batches*BATCH_SIZE

  # Evaluate Model

  model.eval()
  test_loss = 0.0
  test_acc = 0
  test_batches = 0

  for inputs, targets in test_loader:
    one_hot_targets = nn.functional.one_hot(targets, num_classes=10).float()
    inputs, targets, one_hot_targets = inputs.to(device), targets.to(device), one_hot_targets.to(device)

    # Forward pass
    outputs = model(inputs)
    loss = loss_fn(outputs, one_hot_targets)

    # Calculate Accuracy
    _, predicted = torch.max(outputs.data, 1)
    test_acc += (predicted == targets).sum().item()
    test_batches += 1
    test_loss += loss.item()

  test_loss /= test_batches
  test_acc /= test_batches*BATCH_SIZE

  print(f'Epoch {i+1}/{EPOCHS} loss: {train_loss:.4f} - acc: {train_acc:0.4f} - val_loss: {test_loss:.4f} - val_acc: {test_acc:0.4f}')

Epoch 1/20 loss: 0.9262 - acc: 0.6959 - val_loss: 0.3666 - val_acc: 0.9286
Epoch 2/20 loss: 0.8669 - acc: 0.7173 - val_loss: 0.3342 - val_acc: 0.9320
Epoch 3/20 loss: 0.8291 - acc: 0.7302 - val_loss: 0.2941 - val_acc: 0.9352
Epoch 4/20 loss: 0.7987 - acc: 0.7393 - val_loss: 0.2875 - val_acc: 0.9394
Epoch 5/20 loss: 0.7923 - acc: 0.7384 - val_loss: 0.2707 - val_acc: 0.9409
Epoch 6/20 loss: 0.7806 - acc: 0.7423 - val_loss: 0.2621 - val_acc: 0.9444
Epoch 7/20 loss: 0.7725 - acc: 0.7437 - val_loss: 0.2724 - val_acc: 0.9421
Epoch 8/20 loss: 0.7658 - acc: 0.7475 - val_loss: 0.2567 - val_acc: 0.9455
Epoch 9/20 loss: 0.7593 - acc: 0.7480 - val_loss: 0.2562 - val_acc: 0.9463
Epoch 10/20 loss: 0.7592 - acc: 0.7493 - val_loss: 0.2465 - val_acc: 0.9489
Epoch 11/20 loss: 0.7475 - acc: 0.7522 - val_loss: 0.2366 - val_acc: 0.9487
Epoch 12/20 loss: 0.7476 - acc: 0.7517 - val_loss: 0.2281 - val_acc: 0.9497
Epoch 13/20 loss: 0.7411 - acc: 0.7541 - val_loss: 0.2284 - val_acc: 0.9511
Epoch 14/20 loss: 0.7